# Rapport de projet numérique : Modélisation de la sortie d'une foule d'une salle

## Introduction

Le but de ce projet numérique est de modéliser la sortie de plusieurs personnes se trouvant dans une salle.

### Comment éxecuter une simulation

Le plus simple pour faire une simulation est d'éxecuter toute les cellules. Il faut répondre "y" à la  demande d'effacement des variables et "y" à la demande d'effacement des fichiers si une simulation a déjà été faite.

### Importation des librairies

In [110]:
%reset
import itertools
import random
import numpy.random as rd
import numpy as np
import matplotlib.animation as anim
import matplotlib.pyplot as plt
import os
from matplotlib.artist import Artist

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


### Variables d'entrées

Un nombre n de personnes seront placé aléatoirement dans une salle de taille x*y. On veillera à ce que deux personnes ne se superposent pas. Une personne sera représenté par une classe dans laquelle seront stocké sa position, sa vitesse, son rayon, la vitesse maximum qu'il peut atteindre et son numéro.

In [111]:
def initial(n,x,y,poteau,Sim_forces,vm=7,r=0.3):
    """
    Placement aléatoire de n personnes et affectation de leurs caractéristiques
    en prenant en compte les poteaux
    """
    PosVi_tab=rd.rand(n,2,2)
    PosVi_tab[:,0,0]=PosVi_tab[:,0,0]*x+1
    PosVi_tab[:,0,1]=PosVi_tab[:,0,1]*y+1
    classe_tab=[]
    R=[]
    
    for num_part , PosVi in enumerate(PosVi_tab):
        
        ri=np.absolute(np.random.normal(r,0.05))
        v_max=np.absolute(np.random.normal(vm,0.8))
        R.append(ri)
        f=PosVi[1,0]+PosVi[1,1]
        
        if  f > 1 : 
            PosVi[1,0] , PosVi[1,1] = PosVi[1,0]/f , PosVi[1,1]/f
        
        classe_tab=classe_tab+[personne(PosVi[0,0],PosVi[0,1],PosVi[1,0]*v_max,PosVi[1,1]*v_max,ri,v_max,num_part)]
    
    classe_tab=np.array(classe_tab)
     
    #Ajout des personnes poteaux
    classe_tab = np.append(classe_tab,poteau)
    for part in poteau:
        PosVi_tab = np.append(PosVi_tab,[[part.x,part.y],[0,0]])
        PosVi_tab = PosVi_tab.reshape(len(classe_tab),2,2)
        R.append(part.r)
    
    #Replacement des personnes qui se superposent
    ls = np.arange(0,len(classe_tab))
    couple = list(itertools.combinations(ls,2))
    a=1 ; p=0
    
    while a!=0 and p<=100:
        
        a=0
        
        for ij in couple:
            
            i = ij[0]
            j = ij[1]
            #Test de superposition
            if classe_tab[i].superpose(classe_tab[j]):
                
                a=a+1
                classe_tab[i].x = rd.rand()*x+1
                classe_tab[i].y = rd.rand()*y+1
        p=p+1
    
    if p==101 :
        
        input('Trop de personnes dans la salle')
    for num_pers , pers in enumerate(classe_tab):
        
        PosVi_tab[num_pers,0,0]=pers.x
        PosVi_tab[num_pers,0,1]=pers.y
        
    a=0
    if Sim_forces:
        a=0.2
        
    def f1(pers) :
        return pers.y + (pers.r+a) >= 10

    def f2(pers) :
        return pers.y - (pers.r+a) <=0 

    def f3(pers) :
        return pers.x + (pers.r+a) >=10

    def f4(pers) :
        return pers.x - (pers.r+a) <=0

    vect1=np.array([1,0])

    vect2=np.array([1,0])

    vect3=np.array([0,1])

    vect4=np.array([0,1])

    sortie1 = np.array([[5,6],[0,0.5]])
    sortie2 = np.array([[5,6],[9.5,10]])

    mur_class_tab=np.array([mur(f1,vect1,sortie1),mur(f2,vect2),mur(f3,vect3),mur(f4,vect4)])

    return PosVi_tab , classe_tab , R , mur_class_tab

### Variables de sortie

Pour ne pas avoir à garder en mémoire dans des tableaux les positions et vitesses de tout les personnes à chaque pas de temps, ces variables sont écrites dans des fichiers 'tab_pos.txt' et 'tab_vitesse.txt' qui seront lu par le programme lors de l'animation. Ces deux fichiers répertorie donc toutes les variables de sortie de la simulation. Ils sont organisé en deux colonnes: la colonne de gauche contient les positions (ou vitesses) en x des personnes et  la colonne de gauche les variables en y. Chaque ligne correspond à une personne à un pas de temps et chaque saut de ligne correspond à un nouveau pas de temps.

In [112]:
def savefile(tab, name_pos='tab_pos.txt',name_vitesse='tab_vitesse.txt'):
    """
    Sauvegarde le np.array tab dans les fichier name_pos et name_vitesse
    """
    with open(name_pos,'a') as f_pos:
        f_pos.write("\n")
        pos = tab[:,0,:]
        np.savetxt(f_pos,pos)
        
    with open(name_vitesse,'a') as f_vitesse:
        f_vitesse.write("\n")
        vitesse = tab[:,1,:]
        np.savetxt(f_vitesse,vitesse)
        

def readfile(name_pos='tab_pos.txt',name_vitesse='tab_vitesse.txt'):
    """
    Retourne les tableaux contenus dans les tab.txt, sauvegardés avec savefile
    """
    tab_pos = np.loadtxt(name_pos)
    tab_vitesse = np.loadtxt(name_vitesse)
    
    return tab_pos , tab_vitesse

### L'animation

L'animation des personnes se fait grace à la fonction Funcanimation du package matplotlib.animation. Cette fonction nécessite pour fonctionner de la fonction animate qui enlève l'affichage des personnes au pas de temps précédent et affiche les personnes au pas de temps suivant. Cette fonction est présenté ci-dessous. Les personnes sont représneté grâce à la classe Circle de matplotlib.pyplot.
L'animation affiche aussi un chronomètre. Chaque objet text (de matplolib.pyplot) de la liste L représente un pas de temps. Cette liste est créer en dehors de la fonction animate mais leur affichage est géré par celle-ci.
Enfin, les murs sont afficher en dehors de la fonction funcanimation puisque ceci sont fixe durant tout le temps de la simulation.

In [113]:
def animate(i,tab_pos,ax,R,n_liste,L,n_obstacle):
    """
    Fonction qui retire les persones affiché au temps t-dt et affiche celle au temps t
    """
    #Affichage du chronomètre.
    if i!=0 :
        Artist.set_visible(L[i-1], False) # Retire l'affiche du temps précédent
    Artist.set_visible(L[i], True) # Affiche le temps actuel
    #Retire les anciens objets correspondant aux personnes au temps t-dt
    for obj in ax.findobj(match = type(plt.Circle(1, 1))):
        
        obj.remove()
    #Extrait les positions des personnes à afficher
    n_ini = int(np.sum(n_liste[0:i])) + i*n_obstacle
    M=tab_pos[n_ini:n_ini+n_liste[i]+n_obstacle]
    #Affiche un cercle pour représenter la personne. Son centre est la postion de la
    #personne et son rayon est le rayon de la personne.
    for u , j in enumerate(M):
        
        ax.add_artist(plt.Circle(j,radius=R[i][u],fill=False))
        
    return

## I. Les interactions de personne à personne

Les intéraactions entre les personnes constitue le coeur de la modélisation de celle-ci. Plusieurs modèles ont été envisagés et testés nous en présenterons ici deux. Pour coder les intéractions entre les personnes nous nous sommes fortement appuyés sur une classe d'objet : la classe personne. En voici un détail éxplicite.

### A. Définition de la classe personne

La classe personne permet de définir une personne à partir de sa position (x,y), sa vitesse (vx,vy), son rayon (r), sa vitesse maximum (v_max), et le numéro de celle-ci (num). Viennent ensuite quatres méthodes : distance, superpose, distance_sortie et calcul_vect_per.

In [114]:
class personne:
    
    def __init__(self,x,y,vx,vy,r,v_max,num):
        """
        
        Définition de la personne.
        
        Parameters
        ----------
        x : Float
            Coordonée x de la position de la personne.
        y : Float
            Coordonée y de la position de la personne.
        vx : Float
            Coordonée x de la vitesse de la personnes.
        vy : Float
            Coordonée y  de la vitesse de la personne.
        r : Float
            Rayon de la personne (celle-ci à une forme de cercle vu du dessus).
        v_max : Float
            Norme de la vitesse maximale atteignable par la personne.
        num : Int
            Numéro de la personne.

        Returns
        -------
        None.

        """
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.r = r
        self.v_max = v_max
        self.num = num
        
    def distance(self,autre):
        """Retourne la distance entre la personne et une autre, sur l'axe x
        et y"""
        dx = self.x - autre.x
        dy = self.y - autre.y
        return np.sqrt(dx**2+dy**2),dx,dy
    
    def superpose(self,autre):
        """Retourne True si les deux personnes se superposent"""
        
        answer = self.distance(autre)[0] < self.r + autre.r
        return answer
    
    def distances_sortie(self,mur_class_tab):
    
        """
        Retourne la distance et la direction d'une personne à chaque sortie.
        """
        
        d = [] ; v = []
        for num_mur, mur in enumerate(mur_class_tab):
            
            if np.any(mur.sortie != None) == True:
                
                (vecteur,distance) = mur.d_mur(self)
                d.append(distance)  ; v.append(vecteur)
                
        d = np.array(d) ; v=np.array(v)
        
        return d , v

    def calcul_vect_per(self , vect_par):
        
        """Calcul le vecteur perpendicualaire à la direction entre deux personnes
        dont le produit scalaire avec le vecteur vitesse de la personne dont on modifie 
        la vitesse est positif"""
        
        v=np.array([self.vx , self.vy])
        
        if vect_par[1] == 0 :
            
            vect_1 = np.array([0,1])
            vect_2 = np.array([0,-1])
        
        else:
            
            vect_1 = np.array([1,-(vect_par[0]/vect_par[1])])
            vect_2 = np.array([-1,vect_par[0]/vect_par[1]])
            vect_1 = vect_1 / np.linalg.norm(vect_1)
            vect_2 = vect_2 / np.linalg.norm(vect_2)
        
        prod_sca_1 = np.vdot(vect_1 , v)
        
        if prod_sca_1 >= 0 :
            
            return vect_1
        
        return vect_2
    
def change_v_part(vect,pers,PosVi):
    """
    Change la vitesse la particule en lui supprimant sa composante 
    perpendiculaire à un obstacle (mur ou autre peronne).
    """
    
    v = np.array([pers.vx , pers.vy])
    v_prec = PosVi[1,:]
    v_moy = (1/2)*( v_prec + v )
    prod_sca = np.vdot(vect , v )
    prod_sca_moy = np.vdot(vect , v_moy )
    v_nouv = prod_sca * vect
    v_nouv_moy = prod_sca_moy * vect
    
    return v_nouv , v_nouv_moy

### B. Modèle newtonien d'intéraction personne à personne

La première façon de modéliser les intéractions entre personnes est de considérer des forces entre les particules. Dans ce cas les personnes sont modéliser comme des particules subissant des intéractions.

In [115]:
def acceleration(sumF_tab,classe_tab,n):
    
    """Calcul un tableau contenant les accélérations de chaque particules
selon chaque axes. La forme du tableau est donc (n,2)"""
    
    a_tab= np.zeros((n,2))
    for i in range(0,n):
        a_tab[i,:] = sumF_tab[i,:]
        
    return a_tab

def force(part1,part2,detection):
    """ 
    Retourne la force de repulsion entre la particule et une autre.
    Les particule en dehors du rayon de detection sont negligées
    """
    if part1.distance(part2)[0] < detection:
        
        if part1.distance(part2)[1] == 0: #Eviter une division par 0
            Fx = 0
        else:
            Fx = -1/part1.distance(part2)[1]**7 
        
            
        if part1.distance(part2)[2] == 0: #Eviter une division par 0
            Fy = 0
        else:
            Fy = -1/part1.distance(part2)[2]**7 
                
        F = np.array([Fx,Fy])
    
    else :
        F = np.zeros(2)
        
    norm_F=np.linalg.norm(F)
    
    if norm_F>10:
        
        F=F/norm_F
        
    return F

def pos_ini(PosVi_tab,dt):
    pos_ini_tab = np.zeros(np.shape(PosVi_tab))
    pos_ini_tab[:,0,:] = PosVi_tab[:,0,:]-dt*PosVi_tab[:,1,:]
    return pos_ini_tab

def tab_force(classe_tab,detection):
    """
    Retourne le tableau de force entre N particules = len(class_tab). 
    class_tab est le tableau des objets particules. 
    detecttion est le seuil de detection pour la force
    """
    F_tab_x = np.zeros((len(classe_tab),len(classe_tab)))
    F_tab_y = np.zeros((len(classe_tab),len(classe_tab)))
    ls = np.arange(0,len(classe_tab))
    #couple = list(itertools.permutations(ls,2)) 
    couple = list(itertools.combinations(ls,2))
    for ij in couple:
        i = ij[0]
        j = ij[1]
        F_tab_x[i,j] = force(classe_tab[i],classe_tab[j],detection)[0]
        F_tab_x[j,i] = - F_tab_x[i,j]
        F_tab_y[i,j] = force(classe_tab[i],classe_tab[j],detection)[1]
        F_tab_y[j,i] = - F_tab_y[i,j]
    F_tab = np.array([F_tab_x,F_tab_y])
        
    return F_tab

def tab_force_mur(mur_class_tab,classe_tab):
    """
    Retourne le tableau de force sur chaque particule des mur
    tableau de taille (nbr_part,2)
    Reflechir au sens de cette fonction, que ce passe-t-il si la part est au centre ? Elle doit prendre la sortie la plus proche et si elles sont toutes a distances = alors faire un choix aléatoire
    """
    
    F_tab = np.zeros([len(classe_tab),2])
    
    for num_part, part in enumerate(classe_tab):
        #d = np.array([100 * np.ones(len(mur_class_tab)),mur_class_tab])
        #np.reshape(d,(len(mur_class_tab),2))
        d = []
        for num_mur, mur in enumerate(mur_class_tab):
            if np.any(mur.sortie != None) == True:
                pos_sortie = [np.mean(mur.sortie[0]) , np.mean(mur.sortie[1])]
                v = np.array([pos_sortie[0]-part.x,pos_sortie[1]-part.y])
                distance = np.linalg.norm(v)
                d.append([distance,mur])
                
        d = np.array(d)
        i_min = np.argmin(d,axis=0)
                
        mur_proche = d[i_min[0]][1]
        
        F = np.array([mur_proche.force_exit(part)[0],mur_proche.force_exit(part)[1]])
        F_tab[num_part] = F
                
        
    return F_tab

Le problème du modèle newtonien d'intéraction est qu'il est conservatif. On pourra observer lors de sa mise en ouvre que cela ne délivre pas une simulation fidèle. En effet, les personnes se comportent comme des particules d'un gaz soumis au champ gravitationnel terrestre. Cela signçifie que même si elle sont attirées vers le bas le caractère consrvatif garantis que si une particule descend : une autre remonte. Celles-ci ne s'agglomère pas vers le bas or c'est ce phénomène que nous souhaitons observer.

### C. Modèle non-conservatif d'intéraction personne à personne

Nous avons vu les limites du modèle newtonien d'interaction. Pour régler ces problèmes nous avons considérer un modèle non conservatif. Dans celui-ci si deux personnes vont se rentrer dedans alors celle qui s'approche de l'autre (ou les deux si elles s'approchent mutuellement) va perdre la composante de sa vitesse qui est colinéaire au vecteur allant d'une personne à l'autre. Un schéma vaut mieux que des explications.

In [116]:
def change_v_part(vect,pers,PosVi):
        """
        Change la vitesse la particule en lui supprimant sa composante 
        perpendiculaire à un obstacle (mur ou autre peronne).
        """
        
        v = np.array([pers.vx , pers.vy])
        v_prec = PosVi[1,:]
        v_moy = (1/2)*( v_prec + v )
        prod_sca = np.vdot(vect , v )
        prod_sca_moy = np.vdot(vect , v_moy )
        v_nouv = prod_sca * vect
        v_nouv_moy = prod_sca_moy * vect
        
        return v_nouv , v_nouv_moy

## II. Les murs et les sorties

Les intéractions murs/persones sont relativement simple. En effet le mur ne peut pas être modifié par la personne et la personne ne peut pas le franchir. Il y a cependant plusieurs façons de modéliser cette intéraction. Il sera question ici du modèle choisi. Ces dans les murs que se trouvent les sorties physiquement parlant mais aussi dans notre programme car les sorties sont un paramètre de la classe mur qui sera explicité ici. L'intéraction entre les personnes et la sortie est la dernière intéraction à traiter pour finaliser notre modèle.

### A. La classe mur

In [117]:
class mur:
    def __init__(self,f,vect,sortie = None):
        """
        f est la fonction représentative en fonction de x et y du mur
        vect est le vecteur directeur du mur
        sortie est de la forme np.array([[xmin,xmax],[ymin,ymax]])
        """
        self.f = f
        self.vect=vect
        self.sortie = sortie
        
    def collision(self,pers):
        """
        Retourne true si la personne dépasse du mur
        """
        answer = self.f(pers)
        
        return answer
    
    def d_mur(self,particule):
        """
        Mesure la distance entre la particule et la sortie et le vecteur les rejoignant.
        """
        
        pos_sortie = [np.mean(self.sortie[0]) , np.mean(self.sortie[1])]
        v = np.array([pos_sortie[0]-particule.x,pos_sortie[1]-particule.y])
        norm_v = np.linalg.norm(v)
        
        return v , norm_v
    
    def handle_part_exit(self,part):
        """
        Retourne True si la particule sort
        """
        if np.all(self.sortie != None):
            if part.x >= self.sortie[0,0] and part.x <= self.sortie[0,1] and part.y >= self.sortie[1,0] and part.y <= self.sortie[1,1]:
                return True
        else: return False
    
    def force_exit(self,particule):
        """
        Retourne la force qu'exerce la sortie sur une particule
        """
        
        pos_sortie = [np.mean(self.sortie[0]) , np.mean(self.sortie[1])]
        v = np.array([pos_sortie[0]-particule.x,pos_sortie[1]-particule.y])
        norm_v = np.linalg.norm(v)
        
        if norm_v == 0:
            F = np.array([0,0])
            
        else :
            F = 10*v/norm_v
            
        return F

### B. L'intéraction  mur à personne

### C. L'intéraction mur à sortie

In [118]:
def v_sortie(classe_tab,mur_class_tab,n):
    
    """
    Calcul les vitesses de chaque personnes par rapport à la sortie la plus proche.
    """
    
    V=np.zeros((n,2))
    
    for num_part, part in enumerate(classe_tab):
        
        d,v= part.distances_sortie(mur_class_tab)
        i_min=np.argmin(d)
        d=d[i_min]
        v=(v[i_min,:]*part.v_max)/d
        V[num_part] = v
    
    return V

## III. L'intégrateur

### A. L'intégrateur pour le modèle newtonien d'intéraction personne à personne

In [119]:
def change_posvi_forces(PosVi_tab,PosViprec_tab,classe_tab,mur_class_tab, dt,n):
    
    """Calcul les vitesses des particules au nouveau pas de temps"""
    
    F_tab_part = tab_force(classe_tab,100) #Tableau des forces entre particules
    sumF_tab=np.reshape(np.sum(F_tab_part,axis=2),(n,2))
    F_tab_mur = tab_force_mur(mur_class_tab,classe_tab) #Tableau des forces de sortie
    sumF_tab = sumF_tab + F_tab_mur
    a_tab=acceleration(sumF_tab,classe_tab,n)
    V=PosViprec_tab[:,1,:] + 2*a_tab*dt
    Vtot=np.sqrt(V[:,0]**2+V[:,1]**2)
    
    for i , j in enumerate(V) :
        
        vm=classe_tab[i].v_max
        Vtoti=Vtot[i]
        
        if Vtoti > vm :
            
           V[i,:]=(j/Vtoti)*vm
           
    Vmoy = (1/2)*(PosVi_tab[:,1,:]+V)
        
    for i , j in enumerate(classe_tab) :
        
        j.vx = V[i,0]
        j.vy = V[i,1]
        
        for k in mur_class_tab :
            
           if k.collision(j):
              
              V[i,:] , Vmoy[i,:] =change_v_part(k.vect , j , PosVi_tab[i,:,:])
              
    X=PosVi_tab[:,0,:]+dt*Vmoy
    X , V = np.reshape(X,(n,1,2)) , np.reshape(V,(n,1,2))
    PosViprec_tab , PosVi_tab = PosVi_tab , np.concatenate((X,V),axis=1)
    
    for i , j in enumerate(classe_tab):
        
        j.x= PosVi_tab[i,0,0]
        j.y= PosVi_tab[i,0,1]
        j.vx= PosVi_tab[i,1,0]
        j.vy= PosVi_tab[i,1,1]
        
    return PosViprec_tab , PosVi_tab

### B. L'intégrateur pour le modèle non-conservatif d'intéraction personne à personne

In [120]:
def change_posvi(PosVi_tab,classe_tab,mur_class_tab,dt,n):
    
    """Calcul les vitesses et les positions des personnes au nouveau pas de temps"""
    
    #Calcul naif des vitesses et positions au pas de temps suivant
    V=v_sortie(classe_tab,mur_class_tab,n)
    V_moy = (1/2)*(PosVi_tab[:,1,:]+V)
    X=PosVi_tab[:,0,:]+dt*V_moy
    
    #Définition des couples de particules en intéraction
    ls = np.arange(0,len(classe_tab))
    couple = list(itertools.permutations(ls,2))
    
    #Mise à jour des vitesses et positions dans la classe personne
    for num_pers , pers in enumerate(classe_tab):
        
        pers.x= X[num_pers,0]
        pers.y= X[num_pers,1]
        pers.vx= V[num_pers,0]
        pers.vy= V[num_pers,1]
    
    #Décompte des superpositions et du nombre de pas de calcul
    a=1 ; p=0 ; p_max = max(5,int(n/2))
    
    while a!=0 and p<=p_max:
        
        a=0
        
        #Intéraction avec les murs
        for num_pers , pers in enumerate(classe_tab):
            
            c=0
            
            for mur in mur_class_tab :
                
                   if mur.collision(pers) and not mur.handle_part_exit(pers):
                      
                      #Compte le nombre de mur en contact avec la personne
                      c=c+1
                      a=a+1
                      #Change la vitesse de la personne pour que celle-ci ne traverse pas le mur
                      vect = mur.vect
                      (V[num_pers,:] , V_moy[num_pers,:]) = change_v_part(vect,pers,PosVi_tab[num_pers])
        
        #Intéraction de deux personnes entre elles
        random.shuffle(couple)
        for ij in couple:
            
            i = ij[0]
            j = ij[1]
            
            #Test de superposition
            if classe_tab[i].superpose(classe_tab[j]):
                
                a=a+1
                #Calcul du vecteur allant de la personne i à j
                vect_par = PosVi_tab[i,0,:]-PosVi_tab[j,0,:]
                vect_par=vect_par/np.linalg.norm(vect_par)
                
                #Projection de la vitesse sur un vecteur perpendiculaire au vecteur
                #allant de i à j si cette vitesse amène la personne sur l'autre.
                if np.vdot(vect_par,V_moy[i,:])<=0:
                    
                    #Calcul du vecteur perpendiculaire
                    vect_per = classe_tab[i].calcul_vect_per(vect_par)
                    #Projection de la vitesse
                    (V[i,:] , V_moy[i,:]) = change_v_part(vect_per,classe_tab[i],PosVi_tab[i])
                
                elif np.vdot(vect_par,V_moy[j,:])>=0:
                    
                    #Calcul du vecteur perpendiculaire
                    vect_per = classe_tab[j].calcul_vect_per(vect_par)
                    #Projection de la vitesse
                    (V[j,:] , V_moy[j,:]) = change_v_part(vect_per,classe_tab[j],PosVi_tab[j])
                    
        X=PosVi_tab[:,0,:]+dt*V_moy
        p=p+1
        #Mise à jour des vitesses et positions dans la classe personne
        for num_pers , pers in enumerate(classe_tab):
            
            pers.x= X[num_pers,0]
            pers.y= X[num_pers,1]
            pers.vx= V[num_pers,0]
            pers.vy= V[num_pers,1]
    #Mise à jour de PosVi_tab
    X , V = np.reshape(X,(n,1,2)) , np.reshape(V,(n,1,2))
    PosVi_tab = np.concatenate((X,V),axis=1)
    
    return PosVi_tab

## Résultats

In [121]:
#Définition des seed pour la reproductibilité
random.seed(2021)
rd.seed(2021)

#Suppression des anciens fichiers
ans_user = input('Voulez-vous supprimer les anciens fichiers de position ? y/n    ')
if ans_user == 'y':
    os.remove('tab_pos.txt')
    os.remove('tab_vitesse.txt')
    print('Fichiers supprimés.')

#interval de temps
dt = 0.01
#Nombre de particule
n = 50
#Type de simulation
Sim_forces =True # True pour du Newtonien, False sinon
#Initialisation des personnes, des murs et des sorties
#Particule qui seront en fait des obstacles
poteau = np.array([personne(5,5,0,0,1,0,0)])
#Nombre d'obstacle
n_obstacle = len(poteau)
#Particules à proprement parlé, murs et sorties
PosVi_tab , classe_tab , r , mur_class_tab = initial(n,8.5,8.5,poteau,Sim_forces)
#Positions précedentes
if Sim_forces:
    PosViprec_tab = pos_ini(PosVi_tab,dt)
#Sauvegarde des position initiales
savefile(PosVi_tab)

#Définition du temps de fin de simulation
t_fin = 10

#Liste du nombre de personne et du rayon des personnes
n_liste = [n]
R=[r]

#Effectuer la simulation
p=0
while n!=0 and p*dt<=t_fin :
    n_tot = n + n_obstacle
    #Calcul des nouvelles positions
    if Sim_forces:
        PosViprec_tab , PosVi_tab = change_posvi_forces(PosVi_tab,PosViprec_tab,classe_tab,mur_class_tab,dt,n_tot)
    else:
        PosVi_tab = change_posvi(PosVi_tab,classe_tab,mur_class_tab,dt,n_tot)
    #Test de la distance à la sortie
    n_inside = n_liste[-1]
    r_bis=r[:]
    c=0
    for num , part in enumerate(classe_tab):
        for mur in mur_class_tab:
            if mur.handle_part_exit(part) == True and part.v_max!=0: #La personne sort
                n_inside = n_inside - 1
                #Supprime la personne de classe_tab
                classe_tab = np.delete(classe_tab,num-c)
                #Supprime la ligne lié a la position de la personne
                PosVi_tab = np.delete(PosVi_tab,num-c,0)
                if Sim_forces:
                    PosViprec_tab = np.delete(PosViprec_tab,num-c,0)
                #Supprime le rayon de la personne
                r_bis.pop(num-c)
                #Compte le nombre de personne supprimé ce pas de temps
                c=c+1
    #Actualise les listes de rayons et de nombre de particules    
    r=r_bis[:]
    R=R+[r[:]]
    n_liste.append(n_inside)
    n = n_inside
    #Compte le nombre de pas de temps
    p=p+1
    #Sauvegarde des nouvelles positions
    savefile(PosVi_tab)
print('Évacuation en {} s'.format(p*dt))



Voulez-vous supprimer les anciens fichiers de position ? y/n    y
Fichiers supprimés.
Évacuation en 10.01 s


In [122]:
#Représentation graphique

#Transfert du fichier position en un tableau
tab_pos , tab_vitesse = readfile()

#Définition de la figure
fig, ax = plt.subplots()
fig.set_sizes=(10,10)

#Visualisation des murs ?
M=[0,10,10,0,0] ; N =[0,0,10,10,0]

#Définition de l'axe
ax.axis('equal')
ax.set(xlim=(-1, 11), ylim=(-1, 11))

L=[]
for i in range(p):
    s=str(i*dt)
    L=L+[plt.text(11,11,s)]
    Artist.set_visible(L[i], False)

#Fonction d'initialisation
def init():
    pass
    return

#Plot de la figure
plt.plot(M,N)

#Animation
ani = anim.FuncAnimation(fig, animate, frames=p, init_func= init, blit=False,save_count=p, 
                             interval=100, repeat=False , fargs = (tab_pos,ax,R,n_liste,L,n_obstacle))

#Sauvegarde de l'animation en fichier .mp4
#ans_user = input('Sauvegarder animation ? y/n    ')
#if ans_user == 'y':
    #Writer = anim.writers['ffmpeg']
    #writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
    #ani.save('simulation.mp4', writer=writer)
#Représentation graphique

#Transfert du fichier position en un tableau
tab_pos , tab_vitesse = readfile()

#Visualisation des murs
M=[0,10,10,0,0] ; N =[0,0,10,10,0]

#Définition de l'axe
ax.axis('equal')
ax.set(xlim=(-1, 11), ylim=(-1, 11))

L=[]
for i in range(p):
    s=str(i*dt)
    L=L+[plt.text(11,11,s)]
    Artist.set_visible(L[i], False)

#Fonction d'initialisation
def init():
    pass
    return

#Plot de la figure
plt.plot(M,N)

#Animation
ani = anim.FuncAnimation(fig, animate, frames=p, init_func= init, blit=False,save_count=p, 
                             interval=100, repeat=False , fargs = (tab_pos,ax,R,n_liste,L,n_obstacle))

#Sauvegarde de l'animation en fichier .mp4
#ans_user = input('Sauvegarder animation ? y/n    ')
#if ans_user == 'y':
    #Writer = anim.writers['ffmpeg']
    #writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
    #ani.save('simulation.mp4', writer=writer)

<IPython.core.display.Javascript object>